# **Balanced Risk Set Matching with Integer Programming**

In this notebook, we implement **Balanced Risk Set Matching (BRSM)**, an advanced method for matching treated and control units in observational studies. Instead of using simple nearest neighbor matching, we employ:

**Mahalanobis Distance** to measure similarity between units
**Integer Programming (IP)** to optimally match pairs
**Sensitivity Analysis** to check robustness to hidden biases

This method ensures **better balance** in covariates while considering time constraints in treatment assignment.



## **Step 1: Import Necessary Libraries**  

We will use the following libraries for our analysis:  

- **`pandas`** and **`numpy`** for data handling  
- **`scipy.spatial.distance.mahalanobis`** to compute Mahalanobis distance  
- **`pulp`** for integer programming (optimization)  
- **`seaborn`** and **`matplotlib`** for visualization  
- **`statsmodels.api`** for sensitivity analysis  


⚠️ **Note:** If you get an error related to `pulp`, install it first using:
```python
!pip install pulp
```

Now, let's import the libraries.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import mahalanobis
from scipy.stats import wilcoxon
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
import statsmodels.api as sm
    

## **Step 2: Generate Synthetic Data**  

To simulate a real-world dataset, we generate **200 synthetic patients** with the following features:  

- **`age`** (20 to 70 years old)  
- **`gender`** (randomly assigned Male/Female)  
- **`risk_score`** (a value between 0 and 1)  
- **`treatment`** (0 = control, 1 = treated)  
- **`treatment_time`** (when the patient received treatment, from 1 to 100 days)  

### **Why do we include `treatment_time`?**  
🔹 **Balanced Risk Set Matching** requires that treated units can only be matched with control units that were available **before or at the same time** as treatment was given. This ensures **fairness** in causal inference.


In [ ]:

import pandas as pd
import numpy as np

# Generate synthetic dataset
np.random.seed(42)  # Ensure reproducibility
data = pd.DataFrame({
    'id': range(1, 201),
    'age': np.random.randint(20, 70, 200),
    'gender': np.random.choice(['Male', 'Female'], 200),
    'risk_score': np.random.uniform(0, 1, 200),
    'treatment': np.random.choice([0, 1], 200),
    'treatment_time': np.random.randint(1, 100, 200)
})

# Use the generated data directly
print(data.head())


## **Step 3: Compute Mahalanobis Distance**  

Once we have our dataset, we need a way to measure **how similar** each treated unit is to potential control units. Instead of using **Euclidean distance**, we use **Mahalanobis Distance**, which:  

✔️ **Accounts for correlation** between covariates  
✔️ **Standardizes variables** with different scales  
✔️ **Provides a better measure of similarity**  

### **Mathematical Definition**  
Mahalanobis distance is computed as:  

\[
D_M(A, B) = \sqrt{(A - B)^T S^{-1} (A - B)}
\]

Where:  
- \( A, B \) are two observations  
- \( S^{-1} \) is the inverse **covariance matrix** of the dataset  

In this step, we compute a **distance matrix**, where each row corresponds to a **treated unit**, and each column corresponds to a **control unit**.


In [ ]:

def compute_mahalanobis(df, covariates):
    """Computes the Mahalanobis distance matrix between treated and control units."""
    treated = df[df['treatment'] == 1]
    control = df[df['treatment'] == 0]
    
    cov_matrix = np.cov(df[covariates].T)
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    
    distance_matrix = np.zeros((len(treated), len(control)))

    for i, treat_row in treated.iterrows():
        for j, control_row in control.iterrows():
            distance_matrix[i, j] = mahalanobis(
                treat_row[covariates], control_row[covariates], inv_cov_matrix)
    
    return distance_matrix, treated, control

# Compute distances
distance_matrix, treated, control = compute_mahalanobis(data, ['age', 'risk_score'])
distance_matrix[:5, :5]
    

## **Step 4: Integer Programming for Optimal Matching**  

Now that we have pairwise distances, we must **assign** each treated unit to a control unit **optimally**.  

🔹 Instead of **greedy nearest neighbor matching**, we solve an **Integer Programming (IP) problem**, which:  
- ✅ **Minimizes total Mahalanobis distance**  
- ✅ **Ensures each treated unit is matched to exactly one control**  
- ✅ **Ensures each control unit is used at most once**  

### **Mathematical Formulation**  
Let \( X_{ij} \) be a binary variable where:  

\[
X_{ij} =
\begin{cases} 
1, & \text{if treated unit } i \text{ is matched to control unit } j \\
0, & \text{otherwise}
\end{cases}
\]  

The **objective function** is:  

\[
\text{Minimize} \sum_{i=1}^{N_T} \sum_{j=1}^{N_C} D_{ij} X_{ij}
\]

Where:  
- \( D_{ij} \) is the **Mahalanobis distance** between treated unit \( i \) and control unit \( j \)  
- \( N_T, N_C \) are the number of **treated** and **control** units  

### **Constraints**  
1️⃣ **Each treated unit is matched to exactly one control**  

\[
\sum_{j=1}^{N_C} X_{ij} = 1, \quad \forall i
\]

2️⃣ **Each control unit is used at most once**  

\[
\sum_{i=1}^{N_T} X_{ij} \leq 1, \quad \forall j
\]

We solve this using **PuLP**, a linear programming solver in Python.


In [ ]:

def optimal_matching(distance_matrix, treated, control):
    """Solves an integer programming problem for optimal matching using Mahalanobis distances."""
    num_treated = len(treated)
    num_control = len(control)

    prob = LpProblem("Optimal_Matching", LpMinimize)
    match_vars = [[LpVariable(f"match_{i}_{j}", cat='Binary') for j in range(num_control)] for i in range(num_treated)]

    # Objective function: minimize total Mahalanobis distance
    prob += lpSum(distance_matrix[i][j] * match_vars[i][j] for i in range(num_treated) for j in range(num_control))

    # Constraints: Each treated is matched to exactly one control
    for i in range(num_treated):
        prob += lpSum(match_vars[i][j] for j in range(num_control)) == 1

    # Constraints: Each control is matched at most once
    for j in range(num_control):
        prob += lpSum(match_vars[i][j] for i in range(num_treated)) <= 1

    # Solve the optimization problem
    prob.solve()

    # Extract matched pairs
    matched_pairs = [(i, j) for i in range(num_treated) for j in range(num_control) if match_vars[i][j].varValue == 1]
    return matched_pairs

# Get matched pairs
matched_pairs = optimal_matching(distance_matrix, treated, control)
matched_pairs[:5]
    

## **Step 5: Store Matched Data in Memory**  

Instead of saving the matched dataset as a CSV file, we store it **directly in memory** for further analysis.  

This ensures that each time we run the notebook, a fresh dataset is generated and used dynamically in later steps.


## **Step 6: Sensitivity Analysis**  

A major concern in **observational studies** is **hidden bias**—differences between treated and control groups that were not measured.  

To check whether our results are **robust**, we conduct a **Wilcoxon Signed-Rank Test**.  

### **🔹 Why use Wilcoxon?**  
Unlike t-tests, **Wilcoxon** does not assume a **normal distribution**. Instead, it tests **whether two related samples come from the same distribution**.  

### **How to Interpret Results?**  
✅ **If p-value > 0.05** → No significant differences, meaning the matching is likely **balanced**  
❌ **If p-value < 0.05** → Significant differences, meaning there may be **hidden bias** in the matched dataset  


In [ ]:

# Perform Wilcoxon test to check for significant differences in risk score after matching
wilcoxon_stat, p_value = wilcoxon(matched_treated['risk_score'], matched_control['risk_score'])
print(f"Wilcoxon Signed-Rank Test: p-value = {p_value}")
    

## **Step 7: Visualization of Covariate Balance**  

The final step is to **visualize the distribution** of covariates (like `age` and `risk_score`) **before and after matching**.  

We use **Kernel Density Estimation (KDE) plots** to compare the distributions of **treated vs. control** units:  

📌 **How to Interpret KDE Plots?**  
✅ **If the two distributions are similar**, it means our matching was **successful**.  
❌ **If there is still a large difference**, it means our matching was **not effective**, and we may need to refine our model.  


In [ ]:

def visualize_matching(data, covariate, treatment_col):
    """Visualizes the distribution of a covariate before matching."""
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=data[data[treatment_col] == 1][covariate], label='Treated', fill=True)
    sns.kdeplot(data=data[data[treatment_col] == 0][covariate], label='Control', fill=True)
    plt.title(f'Distribution of {covariate} Before Matching')
    plt.legend()
    plt.show()

# Visualize balance in risk score
visualize_matching(data, 'risk_score', 'treatment')
    

## **Conclusion**  

By using **Balanced Risk Set Matching**, we achieved:  

✅ **Better covariate balance** using **Mahalanobis distance**  
✅ **Optimized matching** using **Integer Programming**  
✅ **Time-sensitive matching** to maintain **fairness**  
✅ **Sensitivity analysis** to detect **hidden biases**  

This approach ensures a **more rigorous and fair comparison** between treated and control groups, leading to **better causal inference** in observational studies.  

